<center><h2><strong><font color="blue">Social Network Analysis (SNA)</font></strong></h2></center>
<center><h3><strong><font color="blue"><a href="https://taudata.blogspot.com">https://taudata.blogspot.com</a></font></strong></h3></center>

<img alt="" src="https://github.com/taudataid/eLearning/blob/master/images/covers/taudata-cover.jpg?raw=1"/>

<center><h2><strong><font color="blue">NLPTM-05: Pendahuluan Representasi Teks/Dokumen (VSM & Word Embedding)</font></strong></h2></center>
<center><h3><strong><font color="blue"><a href="https://taudata.blogspot.com/2022/04/nlptm-05.html">https://taudata.blogspot.com/2022/04/nlptm-05.html</a></font></strong></h3></center>
<b><center><h3>(C) Taufik Sutanto</h3></center>

In [ ]:
# Installing Modules for Google Colab
import warnings, nltk; warnings.simplefilter('ignore')

try:
    import google.colab; IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataNlpTm.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/Tweets_example.json
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger

    !pip install spacy python-crfsuite unidecode textblob sastrawi tweepy twython
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm

    nltk.download('popular')
    print("Running the code in colab, don't forget to change the runtime (if needed)")    
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded


In [ ]:
import taudataNlpTm as tau, seaborn as sns; sns.set()
import tweepy, json, nltk, urllib.request
from textblob import TextBlob
from nltk.tokenize import TweetTokenizer
from gensim.models import Word2Vec, FastText
from tqdm import tqdm_notebook as tqdm

# pertama-tama mari kita Load Data twitter dari pertemuan sebelumnya

* Silahkan gunakan data baru (crawl lagi) jika diinginkan

In [ ]:
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [ ]:
# karena ToS data json ini dikirimkan terpisah hanya untuk kalangan terbatas.

T2 = loadTweets(file='data/Tweets_example.json')
print('Total data = {}'.format(len(T2)))
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

Total data = 10000
tweet pertama oleh "Nuy_indra" : "RT @aiceindonesia: Semoga dengan adanya aksi sosial ini masyarakat Indonesia makin disiplin memakai masker, patuhi protokol kesehatan, dan…"


In [ ]:
# Contoh mengambil hanya data tweet
data = [t['full_text'] for t in T2]
data[:5] # 5 tweet pertama

['RT @aiceindonesia: Semoga dengan adanya aksi sosial ini masyarakat Indonesia makin disiplin memakai masker, patuhi protokol kesehatan, dan…',
 'RT @ecosocrights: Bukan hanya konglomerat, kekayaan para pejabat juga naik selama pandemi, termasuk kekayaan Presiden. Pandemi menguntungka…',
 'siapa diantara temen temen disini yg kekayaannya selama pandemi naik 70%?',
 'RT @Gamal_Albinsaid: Tercatat belasan ribu anak menjadi yatim piatu karena pandemi COVID-19.\n\nIni memotivasi saya mengembangkan program Ana…',
 'RT @kyumbin131: ✨ 22nd Junkyu Birthday Project 🇮🇩 ✨\nPenyaluran 22 paket sembako utk keluarga/masyarakat yg kesulitan di masa pandemi.\n\nSemo…']

# PreProcessing Data Text-nya

In [ ]:
# pre processing

stops, lemmatizer = tau.LoadStopWords(lang='id')
stops.add('rt')
stops.add('..')
for i,d in tqdm(enumerate(data)):
    data[i] = tau.cleanText(d, lemma=lemmatizer, stops = stops, symbols_remove = True, min_charLen = 2)
print(data[0])

0it [00:00, ?it/s]

aiceindonesia moga aksi sosial masyarakat indonesia disiplin pakai masker patuh protokol sehat


In [ ]:
# Menggunakan modul SciKit untuk merubah data tidak terstruktur ke VSM
# Scikit implementation http://scikit-learn.org/stable/modules/feature_extraction.html

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

In [ ]:
# VSM - "binari"
binary_vectorizer = CountVectorizer(binary = True)
binari = binary_vectorizer.fit_transform(data)
binari.shape # ukuran VSM

(10000, 9851)

In [ ]:
# Sparse vectors/matrix
binari[0]

<1x9851 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [ ]:
# Mengakses Datanya
print(binari[0].data)
print(binari[0].indices)

[1 1 1 1 1 1 1 1 1 1 1 1]
[ 141 5623  186 8381 5299 3493 2048 6503 5279 6644 7158 7909]


In [ ]:
# Kolom dan term
print(str(binary_vectorizer.vocabulary_)[:93])

{'aiceindonesia': 141, 'moga': 5623, 'aksi': 186, 'sosial': 8381, 'masyarakat': 5299, 'indone


In [ ]:
# VSM term Frekuensi : "tf"
tf_vectorizer = CountVectorizer(binary = False)
tf = tf_vectorizer.fit_transform(data)

print(tf.shape) # Sama
print(tf[0].data) # Hanya data ini yg berubah
print(tf[0].indices) # Letak kolomnya tetap sama

(10000, 9851)
[1 1 1 1 1 1 1 1 1 1 1 1]
[ 141 5623  186 8381 5299 3493 2048 6503 5279 6644 7158 7909]


In [ ]:
d = tf_vectorizer.vocabulary_
kata_kolom = {k:v for v,k in d.items()}
kata_kolom[597]

'awkward'

In [ ]:
# VSM term Frekuensi : "tf-idf"
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(data)

print(tfidf.shape) # Sama
print(tfidf[0].data) # Hanya data ini yg berubah
print(tfidf[0].indices) # Letak kolomnya berbeda, namun jumlah kolom dan elemennya tetap sama

(10000, 9851)
[0.24587303 0.30198705 0.33266445 0.27506537 0.29841056 0.31648834
 0.21882555 0.19953934 0.25019267 0.3566293  0.23883859 0.37186648]
[7909 7158 6644 5279 6503 2048 3493 5299 8381  186 5623  141]


In [ ]:
# VSM term Frekuensi : "tf-idf"
tfidf_vectorizer = TfidfVectorizer(smooth_idf= False, sublinear_tf=True)
tfidf = tfidf_vectorizer.fit_transform(data)
print(tfidf.shape) # Sama
print(tfidf[0].data) # Hanya data ini yg berubah
print(tfidf[0].indices) # Letak kolomnya = tfidf

(10000, 9851)
[0.24525912 0.3016796  0.3328828  0.27454194 0.29806418 0.31638117
 0.21820422 0.19893964 0.24958561 0.3576212  0.2382173  0.37361798]
[7909 7158 6644 5279 6503 2048 3493 5299 8381  186 5623  141]


In [ ]:
# Frequency Filtering di VSM
tfidf_vectorizer = TfidfVectorizer()
tfidf_1 = tfidf_vectorizer.fit_transform(data)

tfidf_vectorizer = TfidfVectorizer(max_df=0.75, min_df=5)
tfidf_2 = tfidf_vectorizer.fit_transform(data)

print(tfidf_1.shape)
print(tfidf_2.shape)

(10000, 9851)
(10000, 2382)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True, smooth_idf= True, sublinear_tf=True, 
                                   ngram_range=(1, 2), max_df=0.90, min_df=2)

tfidf_3 = tfidf_vectorizer.fit_transform(data)
print(tfidf_3.shape)

(10000, 12312)


<h2 id="Best-Match-Formula-:-BM25">Best-Match Formula : BM25</h2>

<p><img alt="" src="https://github.com/taudataid/eLearning/blob/master/images/3_bm25_simple.png?raw=1" style="height: 123px; width: 300px;" /></p>

<ol>
	<li>di IR nilai b dan k yang optimal adalah :&nbsp;<strong> <em>b</em> = 0.75&nbsp; dan k = [1.2 - 2.0]&nbsp; &nbsp;</strong><br />
	ref:&nbsp;<em>Christopher, D. M., Prabhakar, R., &amp; Hinrich, S. C. H. &Uuml;. T. Z. E. (2008). Introduction to information retrieval.&nbsp;An Introduction To Information Retrieval,&nbsp;151, 177.</em></li>
	<li>Tapi kalau untuk TextMining (clustering) nilai <strong>k optimal adalah 20, nilai b = sembarang (boleh = 0.75)</strong><br />
	ref:&nbsp;<em>Whissell, J. S., &amp; Clarke, C. L. (2011). Improving document clustering using Okapi BM25 feature weighting.&nbsp;Information retrieval,&nbsp;14(5), 466-487.</em></li>
	<li><strong>avgDL </strong>adalah rata-rata panjang dokumen di seluruh dataset dan <strong>DL </strong>adalah panjang dokumen D.<br />
	hati-hati, ini berbeda dengan &nbsp;tf-idf MySQL diatas.</li>
</ol>


In [ ]:
# Variasi pembentukan matriks VSM:
d1 = '@udin76, Minum kopi pagi-pagi sambil makan pisang goreng is the best'
d2 = 'Belajar NLP dan Text Mining ternyata seru banget sadiezz'
d3 =  'Sudah lumayan lama bingits tukang Bakso belum lewat'
d4 = 'Aduh ga banget makan Mie Ayam p4k4i kesyap, please deh'

D = [d1, d2, d3, d4]
# Jika kita menggunakan cara biasa: 
tfidf_vectorizer = TfidfVectorizer()
vsm = tfidf_vectorizer.fit_transform(D)
print(tfidf_vectorizer.vocabulary_)

{'udin76': 35, 'minum': 21, 'kopi': 14, 'pagi': 24, 'sambil': 28, 'makan': 18, 'pisang': 25, 'goreng': 11, 'is': 12, 'the': 33, 'best': 6, 'belajar': 4, 'nlp': 22, 'dan': 8, 'text': 32, 'mining': 20, 'ternyata': 31, 'seru': 29, 'banget': 3, 'sadiezz': 27, 'sudah': 30, 'lumayan': 17, 'lama': 15, 'bingits': 7, 'tukang': 34, 'bakso': 2, 'belum': 5, 'lewat': 16, 'aduh': 0, 'ga': 10, 'mie': 19, 'ayam': 1, 'p4k4i': 23, 'kesyap': 13, 'please': 26, 'deh': 9}


In [ ]:
# N-Grams VSM
# Bermanfaat untuk menangkap frase kata, misal: "ga banget", "pisang goreng", dsb
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vsm = tfidf_vectorizer.fit_transform(D)
print(tfidf_vectorizer.vocabulary_)

{'udin76': 69, 'minum': 41, 'kopi': 27, 'pagi': 47, 'sambil': 55, 'makan': 34, 'pisang': 50, 'goreng': 21, 'is': 23, 'the': 65, 'best': 13, 'udin76 minum': 70, 'minum kopi': 42, 'kopi pagi': 28, 'pagi pagi': 48, 'pagi sambil': 49, 'sambil makan': 56, 'makan pisang': 36, 'pisang goreng': 51, 'goreng is': 22, 'is the': 24, 'the best': 66, 'belajar': 9, 'nlp': 43, 'dan': 16, 'text': 63, 'mining': 39, 'ternyata': 61, 'seru': 57, 'banget': 6, 'sadiezz': 54, 'belajar nlp': 10, 'nlp dan': 44, 'dan text': 17, 'text mining': 64, 'mining ternyata': 40, 'ternyata seru': 62, 'seru banget': 58, 'banget sadiezz': 8, 'sudah': 59, 'lumayan': 32, 'lama': 29, 'bingits': 14, 'tukang': 67, 'bakso': 4, 'belum': 11, 'lewat': 31, 'sudah lumayan': 60, 'lumayan lama': 33, 'lama bingits': 30, 'bingits tukang': 15, 'tukang bakso': 68, 'bakso belum': 5, 'belum lewat': 12, 'aduh': 0, 'ga': 19, 'mie': 37, 'ayam': 2, 'p4k4i': 45, 'kesyap': 25, 'please': 52, 'deh': 18, 'aduh ga': 1, 'ga banget': 20, 'banget makan': 7

In [ ]:
# Vocabulary based VSM
# Bermanfaat untuk menghasilkan hasil analisa yang "bersih"
# variasi 2
d1 = '@udin76, Minum kopi pagi-pagi sambil makan pisang goreng is the best'
d2 = 'Belajar NLP dan Text Mining ternyata seru banget sadiezz'
d3 =  'Sudah lumayan lama bingits tukang Bakso belum lewat seru'
d4 = 'Aduh ga banget makan Mie Ayam p4k4i kesyap, please deh'
D = [d1,d2,d3,d4]
Vocab = {'seru banget':0, 'seru':1, 'the best':2, 'lama':3, 'text mining':4, 'nlp':5, 'ayam':6}
tf_vectorizer = CountVectorizer(binary = False, vocabulary=Vocab)
tf = tf_vectorizer.fit_transform(D)
print(tf.toarray())
tf_vectorizer.vocabulary_

[[0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0]
 [0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1]]


{'seru banget': 0,
 'seru': 1,
 'the best': 2,
 'lama': 3,
 'text mining': 4,
 'nlp': 5,
 'ayam': 6}

In [ ]:
Vocab = {'seru banget':0, 'the best':1, 'lama':2, 'text mining':3, 'nlp':4, 'ayam':5}
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase=True, vocabulary=Vocab)
vsm = tfidf_vectorizer.fit_transform(D)
print(tfidf_vectorizer.vocabulary_)
# VSM terurut sesuai definisi dan terkesan lebih "bersih"
# Perusahaan besar biasanya memiliki menggunakan teknik ini dengan vocabulary yang comprehensif
# Sangat cocok untuk Sentiment Analysis

{'seru banget': 0, 'the best': 1, 'lama': 2, 'text mining': 3, 'nlp': 4, 'ayam': 5}


In [ ]:
data[:3]

['aiceindonesia moga aksi sosial masyarakat indonesia disiplin pakai masker patuh protokol sehat',
 'ecosocrights konglomerat kaya jabat pandemi kaya presiden pandemi menguntungka',
 'temen temen yg kaya pandemi']

In [ ]:
# Rubah bentuk data seperti yang dibutuhkan genSim
# Bisa juga dilakukan dengan memodifikasi fungsi "cleanText" (agar lebih efisien)

data_we = []
for doc in data:
    Tokens = [str(w) for w in TextBlob(doc).words]
    data_we.append(Tokens)
print(data_we[:3])

[['aiceindonesia', 'moga', 'aksi', 'sosial', 'masyarakat', 'indonesia', 'disiplin', 'pakai', 'masker', 'patuh', 'protokol', 'sehat'], ['ecosocrights', 'konglomerat', 'kaya', 'jabat', 'pandemi', 'kaya', 'presiden', 'pandemi', 'menguntungka'], ['temen', 'temen', 'yg', 'kaya', 'pandemi']]


In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html
# train word2vec dengan data di atas

L = 300 # Jumlah neurons = ukuran vektor = jumlah kolom
model_wv = Word2Vec(data_we, min_count=2, vector_size=L, window = 5, workers= -2)
# min_count adalah jumlah kata minimal yang muncul di corpus
# "vector_size" adalah Dimensionality of the word vectors 
# (menurut beberapa literature untuk text disarankan 300-500)
# "window" adalah jarak maximum urutan kata yang di pertimbangkan
# workers = jumlah prosesor yang digunakan untuk menjalankan word2vec
print('Done!...')

Done!...


In [ ]:
# di data yang sebenarnya (i.e. besar) Gensim sering membutuhkan waktu cukup lama
# Untungnya kita bisa menyimpan dan me-load kembali hasil perhitungan model word2vec, misal
model_wv.save('data/model_w2v')
model_wv = Word2Vec.load('data/model_w2v')
print('Done!...')

Done!...


### Hati-hati, Word2vec menggunakan Matriks Dense

<p>Penggunaan memory oleh Gensim kurang lebih sebagai berikut:</p>

<p>Jumlah kata x &quot;size&quot; x 12 bytes</p>

<p>Misal terdapat 100 000 kata unik dan menggunakan 200 layers, maka penggunaan memory =&nbsp;</p>

<p>100,000x200x12 bytes = ~229MB</p>

<p>Jika jumlah size semakin banyak, maka jumlah training data yang diperlukan juga semakin banyak, namun model akan semakin akurat.</p>


In [ ]:
# Melihat vector suatu kata
vektor = model_wv.wv.__getitem__(['psbb'])
print(len(vektor[0])) # Panjang vektor keseluruhan = jumlah neuron yang digunakan
print(vektor[0][:5]) # 5 elemen pertama dari vektornya

300
[-0.00139513 -0.00099477  0.00258718  0.00241652 -0.00276821]


In [ ]:
# Mencari kata terdekat menurut data training dan Word2Vec
model_wv.wv.most_similar('psbb')

[('kontannews', 0.21803781390190125),
 ('muhammad', 0.2029067873954773),
 ('bela', 0.19181834161281586),
 ('biang', 0.1762441098690033),
 ('stiap', 0.17516076564788818),
 ('teori', 0.1729869544506073),
 ('manajemen', 0.1714370846748352),
 ('mensos', 0.16922974586486816),
 ('aktivitas', 0.16919642686843872),
 ('ppkn', 0.16815714538097382)]

In [ ]:
# Melihat similarity antar kata
print(model_wv.wv.similarity('psbb', 'corona'))
print(model_wv.wv.similarity('psbb', 'bioskop'))
print(model_wv.wv.similarity('psbb', 'psbb'))

-0.062678315
-0.039310396
1.0


## Hati-hati Cosine adalah similarity bukan distance
Hal ini akan mempengaruhi interpretasi

In [ ]:
# error jika kata tidak ada di training data
# beckman bukan beckmans ==> hence di Word Embedding PreProcessing harus thourough

kata = 'copid'
try:
    print(model_wv.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')
# ini salah satu kelemahan Word2Vec

error! kata " copid " tidak ada di training data


## Tips:

<p>Hati-hati GenSim tidak menggunakan seluruh kata di training data!.</p>

<p>Perintah berikut akan menghasilkan kata-kata yang terdapat di vocabulary GenSim</p>


In [ ]:
sampel_ = 33
model_wv.wv.index_to_key[:sampel_]

['pandemi',
 'yg',
 'kaya',
 'bantu',
 'rp',
 'miliar',
 'harta',
 'dampak',
 'ga',
 'luhut',
 'jabat',
 'musisi',
 'pandjaitan',
 'binsar',
 'ekonomi',
 'kuliah',
 'oposisicerdas',
 'ya',
 'jokowi',
 'jatengopini',
 'orang',
 'jawa',
 'halo',
 'lho',
 'masyarakat',
 'kreatif',
 'gak',
 'manggung',
 'tangis',
 'tapi',
 'anies',
 'rakyat',
 'turun']

## Hati-hati menginterpretasikan hasil Word2Vec

<h3 id=" FastText-(Facebook-2016)">&nbsp;FastText (Facebook-2016)</h3>

<ul>
	<li>Menggunakan Sub-words: app, ppl, ple - apple</li>
	<li>Paper:&nbsp;https://arxiv.org/abs/1607.04606&nbsp;&nbsp;</li>
	<li>Website:&nbsp;https://fasttext.cc/</li>
	<li>Source:&nbsp;https://github.com/facebookresearch/fastText&nbsp;</li>
</ul>


In [ ]:
# Caution penggunaan memory besar, bila timbul "Memory Error" kecilkan nilai L

L = 100 # Jumlah neurons = ukuran vektor = jumlah kolom
model_FT = FastText(data_we, vector_size=L, window=5, min_count=2, workers=-2)
'Done'

'Done'

In [ ]:
# Mencari kata terdekat menurut data training dan Word2Vec
model_FT.wv.most_similar('psbb')

[('naek', 0.34502407908439636),
 ('mek', 0.3286518454551697),
 ('home', 0.3269801735877991),
 ('bulan', 0.31775084137916565),
 ('blm', 0.3159526288509369),
 ('kapolres', 0.3140672445297241),
 ('himpun', 0.3126278817653656),
 ('psi', 0.3098238706588745),
 ('himperaktif', 0.30772924423217773),
 ('cipedes', 0.30604347586631775)]

In [ ]:
# Melihat similarity antar kata
print(model_FT.wv.similarity('psbb', 'corona'))
print(model_FT.wv.similarity('psbb', 'jakarta'))
print(model_FT.wv.similarity('psbb', 'psbb'))

0.082103185
-0.1547481
1.0


In [ ]:
# Word2Vec VS FastText
try:
    print(model_wv.wv.most_similar('coro'))
except:
    print('Word2Vec error!')
    
try:
    print(model_FT.wv.most_similar('coro'))
except:
    print('FastText error!')

Word2Vec error!
[('corona', 0.5362619161605835), ('coronavirus', 0.4721188545227051), ('narsih', 0.3698681890964508), ('panggil', 0.3449081778526306), ('bersih', 0.3184361159801483), ('coronce', 0.31669026613235474), ('jepit', 0.3149551451206207), ('orgil', 0.3126393258571625), ('sih', 0.3084302246570587), ('siap', 0.3081304430961609)]


# Diskusi:
<ul>
	<li>Apakah kelebihan dan kekurangan WE secara umum?</li>
	<li>Apakah kira-kira aplikasi WE?</li>
	<li>Apakah bisa dijadikan representasi dokumen? Bagaimana caranya?</li>
	<li>Bergantung pada apa sajakah performa model WE?</li>
</ul>

* Preprocessing apa yang sebaiknya dilakukan pada model Word Embedding?
* Apakah Pos Tag bermanfaat disini? Jika iya bagaimana menggunakannya?

<h1>End of Module</h1>
<hr />
<p><img alt="" src="https://github.com/taudataid/eLearning/blob/master/images/meme-cartoon/2_Studying_Linguistic.png?raw=1" style="height:500px; width:667px" /></p>
